# BOSTON HOUSE PRICES DATASET (WORKING TITLE)

In [2]:
## import libraries necessary for the project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston

# DESCRIBE

temporary section guideline notes :
1. use desriptive stats and plots to describe the Boston House Prices dataset.
2. make sure to use only useful and maeningful stats and plots. do not use just fill space 

# INFER

temporary section guideline notes :
1. use inferential stats to analyse whether there is a significant difference in median house prices between houses that are along the Charles river vs those that are not.
2. explain and discuss my findings (ie  

# PREDICT

temporary section guideline notes :
1. use keras to create a neural network that can predict the median house price based on the other variables in the dataset. (Feel free to interpret this as I wish. ie. use all the other variables or use a subset.)

In [4]:
boston = load_boston()
print(boston.data)


[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
